In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
main_dir = 'drive/MyDrive/23Spring-MLMA/isolated/'
os.listdir(main_dir)

['ReadMe',
 '002',
 '005',
 '004',
 '003',
 '001',
 '006',
 'XGboost0.pkl',
 'XGboost1.pkl',
 'XGboost2.pkl',
 'XGboost3.pkl',
 'XGboost4.pkl',
 'XGboost5.pkl']

In [5]:
# get data
def get_original_feature_label(path):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        a = np.swapaxes(a,0,1)
        feature.append(a)
        if i.endswith("001.csv"):
            label.append(0)
        elif i.endswith("002.csv"):
            label.append(1)
        elif i.endswith("003.csv"):
            label.append(2)
        elif i.endswith("004.csv"):
            label.append(3)
        elif i.endswith("005.csv"):
            label.append(4)
        elif i.endswith("006.csv"):
            label.append(5)
        elif i.endswith("007.csv"):
            label.append(6)
        elif i.endswith("008.csv"):
            label.append(7)
        elif i.endswith("009.csv"):
            label.append(8)
        elif i.endswith("010.csv"):
            label.append(9)
        elif i.endswith("011.csv"):
            label.append(10)
        elif i.endswith("012.csv"):
            label.append(11)
    
    return feature,label

# Split on patient level - User-independent


> 6-fold cross-validation





In [ ]:
import xgboost as xgb
def train(train_feature,train_label,test_feature,test_label,num):
    # 定义模型
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=12, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)

    # 预测测试集
    y_pred = model.predict(test_feature)

    # 计算准确率
    accuracy = (y_pred == test_label).sum() / len(test_label)
    print('Accuracy:', accuracy)

    # 保存模型
    pickle.dump(model, open(main_dir+"XGboost"+str(num)+".pkl", 'wb'))

    return accuracy

In [ ]:
patient_all = ["001",'002','003','004','005','006']
acc = []
for j in range(6):
  patient = [patient_all[j]]
  test_feature = []
  test_label = []
  for i in patient:
      f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
      for x in f:
          test_feature.append(x)         
      for x in l:
          test_label.append(x)
  print(patient)
  print(len(test_feature))
  test_feature = np.reshape(test_feature,(np.shape(test_feature)[0],-1))

  patient = [x for x in patient_all if x != patient_all[j]]
  print(patient)
  train_feature = []
  train_label = []
  for i in patient:
      f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
      for x in f:
          train_feature.append(x)         
      for x in l:
          train_label.append(x)
  print(len(train_feature))
  train_feature = np.reshape(train_feature,(np.shape(train_feature)[0],-1))
  acc.append(train(train_feature,train_label,test_feature,test_label,j))
  print(acc)

['001']
120
['002', '003', '004', '005', '006']
604
Accuracy: 0.6
[0.6]
['002']
120
['001', '003', '004', '005', '006']
604
Accuracy: 0.7916666666666666
[0.6, 0.7916666666666666]
['003']
122
['001', '002', '004', '005', '006']
602
Accuracy: 0.5409836065573771
[0.6, 0.7916666666666666, 0.5409836065573771]
['004']
121
['001', '002', '003', '005', '006']
603
Accuracy: 0.5867768595041323
[0.6, 0.7916666666666666, 0.5409836065573771, 0.5867768595041323]
['005']
120
['001', '002', '003', '004', '006']
604
Accuracy: 0.6916666666666667
[0.6, 0.7916666666666666, 0.5409836065573771, 0.5867768595041323, 0.6916666666666667]
['006']
121
['001', '002', '003', '004', '005']
603
Accuracy: 0.7355371900826446
[0.6, 0.7916666666666666, 0.5409836065573771, 0.5867768595041323, 0.6916666666666667, 0.7355371900826446]


In [ ]:
print(np.mean(acc))

0.6577718315795812


# User-dependent


> 5-fold cross-validation





In [40]:
# get data
def get_original_feature_label(path, t1, t2):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    if t2==0:
        num = len(os.listdir(path))-2
    else:
        num = 2
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        a = np.swapaxes(a,0,1)
        if (t1==1)&(t2!=0)&((i[8:10]=="01") or (i[8:10]=="02")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==3)&(t2!=0)&((i[8:10]=="03") or (i[8:10]=="04")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==5)&(t2!=0)&((i[8:10]=="05") or (i[8:10]=="06")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==7)&(t2!=0)&((i[8:10]=="07") or (i[8:10]=="08")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue
        if (t1==9)&(t2!=0)&((i[8:10]=="09") or (i[8:10]=="10")):
            feature.append(a)
            if i.endswith("001.csv"):
                label.append(0)
            elif i.endswith("002.csv"):
                label.append(1)
            elif i.endswith("003.csv"):
                label.append(2)
            elif i.endswith("004.csv"):
                label.append(3)
            elif i.endswith("005.csv"):
                label.append(4)
            elif i.endswith("006.csv"):
                label.append(5)
            elif i.endswith("007.csv"):
                label.append(6)
            elif i.endswith("008.csv"):
                label.append(7)
            elif i.endswith("009.csv"):
                label.append(8)
            elif i.endswith("010.csv"):
                label.append(9)
            elif i.endswith("011.csv"):
                label.append(10)
            elif i.endswith("012.csv"):
                label.append(11)
            continue

        if (t1==1)&(t2==0)&((i[8:10]=="01") or (i[8:10]=="02")):
           continue
        if (t1==3)&(t2==0)&((i[8:10]=="03") or (i[8:10]=="04")):
           continue
        if (t1==5)&(t2==0)&((i[8:10]=="05") or (i[8:10]=="06")):
           continue
        if (t1==7)&(t2==0)&((i[8:10]=="07") or (i[8:10]=="08")):
           continue
        if (t1==9)&(t2==0)&((i[8:10]=="09") or (i[8:10]=="10")):
           continue

        if (t2==0):
          feature.append(a)
          if i.endswith("001.csv"):
              label.append(0)
          elif i.endswith("002.csv"):
              label.append(1)
          elif i.endswith("003.csv"):
              label.append(2)
          elif i.endswith("004.csv"):
              label.append(3)
          elif i.endswith("005.csv"):
              label.append(4)
          elif i.endswith("006.csv"):
              label.append(5)
          elif i.endswith("007.csv"):
              label.append(6)
          elif i.endswith("008.csv"):
              label.append(7)
          elif i.endswith("009.csv"):
              label.append(8)
          elif i.endswith("010.csv"):
              label.append(9)
          elif i.endswith("011.csv"):
              label.append(10)
          elif i.endswith("012.csv"):
              label.append(11)
        
    
    return feature,label

In [41]:
import xgboost as xgb
def train(train_feature,train_label,test_feature,test_label,num,p):
    # 定义模型
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=12, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)

    # 预测测试集
    y_pred = model.predict(test_feature)

    # 计算准确率
    accuracy = (y_pred == test_label).sum() / len(test_label)
    print('Accuracy:', accuracy)

    # 保存模型
    pickle.dump(model, open(main_dir+"XGboost_patient_"+str(p)+"model_"+str(num)+".pkl", 'wb'))

    return accuracy

In [44]:
patient_all = ["001",'002','003','004','005','006']
acc_p = []
for j in range(6):

  patient_model = patient_all[j]
  patient_path = str(main_dir+patient_model+"/isolated_strokes/")
  acc = []

  for i in range(1,10,2):
    train_feature = []
    train_label = []
    f,l = get_original_feature_label(patient_path,i,0)
    for x in f:
        train_feature.append(x)         
    for x in l:
        train_label.append(x)
    print(len(train_label))

    test_feature = []
    test_label = []
    f,l = get_original_feature_label(patient_path,i,1)
    for x in f:
        test_feature.append(x)         
    for x in l:
        test_label.append(x)
    print(len(test_label))
    
    test_feature = np.reshape(test_feature,(np.shape(test_feature)[0],-1))
    train_feature = np.reshape(train_feature,(np.shape(train_feature)[0],-1))
    acc.append(train(train_feature,train_label,test_feature,test_label,i,j))
    print(acc)
  acc_p.append(np.mean(acc))

96
24
Accuracy: 0.625
[0.625]
96
24
Accuracy: 0.8333333333333334
[0.625, 0.8333333333333334]
96
24
Accuracy: 0.4583333333333333
[0.625, 0.8333333333333334, 0.4583333333333333]
96
24
Accuracy: 0.875
[0.625, 0.8333333333333334, 0.4583333333333333, 0.875]
96
24
Accuracy: 0.75
[0.625, 0.8333333333333334, 0.4583333333333333, 0.875, 0.75]
96
24
Accuracy: 0.75
[0.75]
96
24
Accuracy: 0.7083333333333334
[0.75, 0.7083333333333334]
96
24
Accuracy: 0.875
[0.75, 0.7083333333333334, 0.875]
96
24
Accuracy: 0.875
[0.75, 0.7083333333333334, 0.875, 0.875]
96
24
Accuracy: 0.7916666666666666
[0.75, 0.7083333333333334, 0.875, 0.875, 0.7916666666666666]
98
24
Accuracy: 0.5833333333333334
[0.5833333333333334]
98
24
Accuracy: 0.5
[0.5833333333333334, 0.5]
98
24
Accuracy: 0.8333333333333334
[0.5833333333333334, 0.5, 0.8333333333333334]
98
24
Accuracy: 0.75
[0.5833333333333334, 0.5, 0.8333333333333334, 0.75]
98
24
Accuracy: 0.7083333333333334
[0.5833333333333334, 0.5, 0.8333333333333334, 0.75, 0.708333333333333

In [45]:
np.mean(acc_p)

0.7342995169082126